In [1]:
import os
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
llm = GoogleGenerativeAI(api_key=os.getenv('GEMINIAPI_API_KEY'), temperature=0.7, model='gemini-2.0-flash')

In [3]:
name = llm.invoke('I want to open a restorant for Indian food. Suggest me only one fancy name for this.')
print(name)

**"The Raj Pavilion"**


In [4]:
from langchain.document_loaders.csv_loader import CSVLoader

In [5]:
loader = CSVLoader(file_path='./codebasics_faqs.csv', source_column='prompt')
data = loader.load()

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings

In [7]:
embedding = HuggingFaceEmbeddings()

In [8]:
e = embedding.embed_documents(['Hello, my name is Prithvi.'])

In [9]:
type(e)

list

In [10]:
len(e[0])

768

In [11]:
e[0][:5]

[0.04142066463828087,
 -0.031617674976587296,
 -0.018779344856739044,
 0.002613516291603446,
 0.03062448650598526]

In [12]:
from langchain.vectorstores import FAISS

In [13]:
vector_db = FAISS.from_documents(documents=data, embedding=embedding)

In [14]:
retriever = vector_db.as_retriever(score_threshold=0.7)

In [16]:
rdocs = retriever.invoke("how about job placement support?")
rdocs

[Document(id='209dfd99-f8bb-4aba-a395-f22ce4725a80', metadata={'source': 'Do you provide any job assistance?', 'row': 11}, page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.'),
 Document(id='fb0b2c37-da11-4422-b2bb-23ad4a8fe6df', metadata={'source': 'Will this course guarantee me a job?', 'row': 33}, page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepar

In [18]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap, RunnableLambda
from langchain.chains import RetrievalQA

In [19]:
prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [20]:
chain = RetrievalQA.from_chain_type(
                            llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs
                            )

In [22]:
chain.invoke('Do you provide job assistance and also do you provide job gurantee?')

{'query': 'Do you provide job assistance and also do you provide job gurantee?',
 'result': 'Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.\n\nWe want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.',
 'source_documents': [Document(id='209dfd99-f8bb-4aba-a395-f22ce4725a80', metadata={'source': 'Do you provide any job assistance?', 'row': 11}, page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.'),
  Document(id='fb0b2c37-da11-4422-b2bb-23ad4a8fe6df', metadata={'source': 'Will this course g

In [23]:
chain("Do you guys provide internship and also do you offer EMI payments?")

{'query': 'Do you guys provide internship and also do you offer EMI payments?',
 'result': 'Yes\nNo',
 'source_documents': [Document(id='5c6f8205-9c60-482a-bd16-99bc6ab6273b', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}, page_content='prompt: Do you provide any virtual internship?\nresponse: Yes'),
  Document(id='95cf7750-88af-4ff9-bde5-7131520e284b', metadata={'source': 'Do we have an EMI option?', 'row': 13}, page_content='prompt: Do we have an EMI option?\nresponse: No'),
  Document(id='209dfd99-f8bb-4aba-a395-f22ce4725a80', metadata={'source': 'Do you provide any job assistance?', 'row': 11}, page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.'),
  Document(id='fb0b2c37-da11-4422-b2bb-23ad4a8fe6df', metadata={'source': 'Will this course guarantee me a job

In [24]:
chain("do you have javascript course?")

{'query': 'do you have javascript course?',
 'result': "I don't know.",
 'source_documents': [Document(id='744f0e7a-ac1b-4345-a9b0-e136d4c447ef', metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}, page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.'),
  Document(id='e04c4845-74d4-4e41-a5b7-d90f14fa8d59', metadata={'source': 'I’m not sure if this course is good enough for me to invest some money. What can I do?', 'row': 20}, page_content='prompt: I’m not sure if this course is good enough for me to invest some money. What can I do?\nresponse: Don’t worry. Many videos in this course are free so watch them to get an idea of the qu